In [2]:
import tensorflow as tf
tf.__version__

'2.2.0'

In [3]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


False

In [4]:
import pandas as pd
import numpy as np

# fix random seed for reproducibility
seed = 42
np.random.seed(seed)
tf.random.set_seed(seed)

In [7]:
text_data = pd.read_csv('../speeches_millercenter_preprocessed.csv')

In [8]:
text_data.head(1)

,Transcript,Summary,President,Source,Date,URL,Word Count,Affiliation,Label
0,34th time speak oval office year soon time wan...,"In this broadcast from the Oval Office, Presid...",Ronald Reagan,National Archives,"January 11, 1989",https://millercenter.org/the-presidency/presid...,3289,Republican,0


In [9]:
# train test split
from sklearn.model_selection import train_test_split
train_corpus, test_corpus, train_label_nums, test_label_nums, train_label_names, test_label_names = train_test_split(text_data['Transcript'],
                                         np.array(text_data['Label']),
                                         np.array(text_data['Affiliation']),
                                         test_size=0.33, random_state=42)
train_corpus.shape, test_corpus.shape

((261,), (130,))

In [10]:
# Tokenize training speeches
t = tf.keras.preprocessing.text.Tokenizer(oov_token='<UNK>')
# fit the tokenizer on the documents
t.fit_on_texts(train_corpus)
t.word_index['<PAD>'] = 0

In [13]:
print(max([(k, v) for k, v in t.word_index.items()], key = lambda x:x[1]), 
      min([(k, v) for k, v in t.word_index.items()], key = lambda x:x[1]), 
      t.word_index['<UNK>'])

('ascend', 15647) ('<PAD>', 0) 1


In [14]:
train_sequences = t.texts_to_sequences(train_corpus)
test_sequences = t.texts_to_sequences(test_corpus)

In [15]:
print("Vocabulary size={}".format(len(t.word_index)))
print("Number of Documents={}".format(t.document_count))

Vocabulary size=15648
Number of Documents=261


In [ ]:
# What does the below do? Limit the vector length? And why do we need it?

In [21]:
MAX_SEQUENCE_LENGTH = 500
# pad dataset to a maximum review length in words
X_train = tf.keras.preprocessing.sequence.pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH)
X_test = tf.keras.preprocessing.sequence.pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)
X_train.shape, X_test.shape

((261, 500), (130, 500))

In [22]:
VOCAB_SIZE = len(t.word_index)
EMBED_SIZE = 300
EPOCHS=100
BATCH_SIZE=128

In [28]:
word2idx = t.word_index
# Word vectors downloaded from https://fasttext.cc/docs/en/english-vectors.html
FASTTEXT_INIT_EMBEDDINGS_FILE = '../wiki-news-300d-1M-subword.vec'


def load_pretrained_embeddings(word_to_index, max_features, embedding_size, embedding_file_path):    
    
    def get_coefs(word,*arr): 
        return word, np.asarray(arr, dtype='float32')
    
    embeddings_index = dict(get_coefs(*row.split(" ")) 
                                for row in open(embedding_file_path, encoding="utf8", errors='ignore') 
                                    if len(row)>100)

    all_embs = np.stack(embeddings_index.values())
    emb_mean, emb_std = all_embs.mean(), all_embs.std()
    embed_size = all_embs.shape[1]

    nb_words = min(max_features, len(word_to_index))
    embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embedding_size))
    
    for word, idx in word_to_index.items():
        if idx >= max_features: 
            continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: 
            embedding_matrix[idx] = embedding_vector

    return embedding_matrix

In [ ]:
# What does exactly the below? It uses the embeddings to create the corpus vector representation?

In [29]:
ft_embeddings = load_pretrained_embeddings(word_to_index=word2idx, 
                                           max_features=VOCAB_SIZE, 
                                           embedding_size=EMBED_SIZE, 
                                           embedding_file_path=FASTTEXT_INIT_EMBEDDINGS_FILE)
ft_embeddings.shape

/opt/anaconda3/envs/nlp_capstone/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3331: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  exec(code_obj, self.user_global_ns, self.user_ns)


(15648, 300)

In [30]:
model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Embedding(VOCAB_SIZE, EMBED_SIZE,
                                    weights=[ft_embeddings],
                                    trainable=True,
                                    input_length=MAX_SEQUENCE_LENGTH))

model.add(tf.keras.layers.Conv1D(filters=256, kernel_size=5, padding='same', activation='relu'))
model.add(tf.keras.layers.MaxPooling1D(pool_size=5))

model.add(tf.keras.layers.Conv1D(filters=128, kernel_size=5, padding='same', activation='relu'))
model.add(tf.keras.layers.MaxPooling1D(pool_size=5))

model.add(tf.keras.layers.Conv1D(filters=64, kernel_size=5, padding='same', activation='relu'))
model.add(tf.keras.layers.MaxPooling1D(pool_size=5))

model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 500, 300)          4694400   
_________________________________________________________________
conv1d (Conv1D)              (None, 500, 256)          384256    
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 100, 256)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 100, 128)          163968    
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 20, 128)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 20, 64)            41024     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 4, 64)             0

In [ ]:
# What's the validation dataset used for?

In [32]:
# Fit the model
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                      patience=3,
                                      restore_best_weights=True,
                                      verbose=1)

model.fit(X_train, train_label_nums, 
#           validation_data=(X_val, val_sentiments),
          epochs=EPOCHS, 
          batch_size=BATCH_SIZE, 
          shuffle=True,
          callbacks=[es],
          verbose=1)

Epoch 1/100
3/3 [==============================] - 1s 239ms/step - loss: 0.6701 - accuracy: 0.6360
Epoch 2/100
3/3 [==============================] - 1s 237ms/step - loss: 0.7416 - accuracy: 0.6398
Epoch 3/100
3/3 [==============================] - 1s 235ms/step - loss: 0.6441 - accuracy: 0.6398
Epoch 4/100
3/3 [==============================] - 1s 244ms/step - loss: 0.6598 - accuracy: 0.6398
Epoch 5/100
3/3 [==============================] - 1s 241ms/step - loss: 0.6542 - accuracy: 0.6398
Epoch 6/100
3/3 [==============================] - 1s 238ms/step - loss: 0.6368 - accuracy: 0.6398
Epoch 7/100
3/3 [==============================] - 1s 245ms/step - loss: 0.6299 - accuracy: 0.6398
Epoch 8/100
3/3 [==============================] - 1s 264ms/step - loss: 0.6230 - accuracy: 0.6398
Epoch 9/100
3/3 [==============================] - 1s 290ms/step - loss: 0.6012 - accuracy: 0.6398
Epoch 10/100
3/3 [==============================] - 1s 254ms/step - loss: 0.5734 - accuracy: 0.6398
Epoch 11/

3/3 [==============================] - 1s 241ms/step - loss: 3.6628 - accuracy: 0.6935
Epoch 28/100
3/3 [==============================] - 1s 288ms/step - loss: 0.2113 - accuracy: 0.9310
Epoch 29/100
3/3 [==============================] - 1s 257ms/step - loss: 1.2725 - accuracy: 0.5785
Epoch 30/100
3/3 [==============================] - 1s 245ms/step - loss: 0.1822 - accuracy: 0.9349
Epoch 31/100
3/3 [==============================] - 1s 250ms/step - loss: 0.1686 - accuracy: 0.9962
Epoch 32/100
3/3 [==============================] - 1s 252ms/step - loss: 0.1942 - accuracy: 0.9962
Epoch 33/100
3/3 [==============================] - 1s 238ms/step - loss: 0.1558 - accuracy: 0.9962
Epoch 34/100
3/3 [==============================] - 1s 243ms/step - loss: 0.1073 - accuracy: 1.0000
Epoch 35/100
3/3 [==============================] - 1s 244ms/step - loss: 0.0539 - accuracy: 0.9962
Epoch 36/100
3/3 [==============================] - 1s 273ms/step - loss: 0.0235 - accuracy: 0.9923
Epoch 37/100


Epoch 54/100
3/3 [==============================] - 1s 238ms/step - loss: 1.4180e-05 - accuracy: 1.0000
Epoch 55/100
3/3 [==============================] - 1s 282ms/step - loss: 1.3748e-05 - accuracy: 1.0000
Epoch 56/100
3/3 [==============================] - 1s 244ms/step - loss: 7.6483e-06 - accuracy: 1.0000
Epoch 57/100
3/3 [==============================] - 1s 236ms/step - loss: 7.3244e-06 - accuracy: 1.0000
Epoch 58/100
3/3 [==============================] - 1s 246ms/step - loss: 8.1446e-06 - accuracy: 1.0000
Epoch 59/100
3/3 [==============================] - 1s 244ms/step - loss: 9.2257e-06 - accuracy: 1.0000
Epoch 60/100
3/3 [==============================] - 1s 244ms/step - loss: 6.7441e-06 - accuracy: 1.0000
Epoch 61/100
3/3 [==============================] - 1s 237ms/step - loss: 4.4597e-06 - accuracy: 1.0000
Epoch 62/100
3/3 [==============================] - 1s 233ms/step - loss: 1.0862e-05 - accuracy: 1.0000
Epoch 63/100
3/3 [==============================] - 1s 263ms/ste

Epoch 80/100
3/3 [==============================] - 1s 276ms/step - loss: 7.8105e-06 - accuracy: 1.0000
Epoch 81/100
3/3 [==============================] - 1s 241ms/step - loss: 2.5264e-06 - accuracy: 1.0000
Epoch 82/100
3/3 [==============================] - 1s 250ms/step - loss: 4.6138e-06 - accuracy: 1.0000
Epoch 83/100
3/3 [==============================] - 1s 248ms/step - loss: 8.4155e-06 - accuracy: 1.0000
Epoch 84/100
3/3 [==============================] - 1s 261ms/step - loss: 3.7395e-06 - accuracy: 1.0000
Epoch 85/100
3/3 [==============================] - 1s 269ms/step - loss: 2.3582e-06 - accuracy: 1.0000
Epoch 86/100
3/3 [==============================] - 1s 305ms/step - loss: 4.2964e-06 - accuracy: 1.0000
Epoch 87/100
3/3 [==============================] - 1s 298ms/step - loss: 5.9490e-06 - accuracy: 1.0000
Epoch 88/100
3/3 [==============================] - 1s 274ms/step - loss: 6.7970e-06 - accuracy: 1.0000
Epoch 89/100
3/3 [==============================] - 1s 243ms/ste

In [44]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
predictions = model.predict_classes(X_test, batch_size=2048, verbose=0).ravel()

print("Accuracy: %.2f%%" % (accuracy_score(test_label_nums.astype(int), predictions)*100))
print(classification_report(test_label_nums, predictions))
pd.DataFrame(confusion_matrix(test_label_nums, predictions))

Accuracy: 63.08%
              precision    recall  f1-score   support

           0       0.57      0.65      0.61        57
           1       0.69      0.62      0.65        73

    accuracy                           0.63       130
   macro avg       0.63      0.63      0.63       130
weighted avg       0.64      0.63      0.63       130



,0,1
0,37,20
1,28,45
